In [1]:
import tensorflow as tf
from random import shuffle

from utils.data_analyze import *
from utils.transform import *
from dataset.test_data_loader import TestDataLoader
from dataset.train_data_loader import TrainDataLoader

/home/messor/.pyenv/versions/3.6.4/envs/shinerio-python3.6.4-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# 生成字符串型的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


# 生成实数型的属性
def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


In [3]:
logger = create_logger(os.path.join(config.log_path, 'train.log'))
logger.info('config:\n{}'.format(pprint.pformat(config)))
id_attribute_dict, classSymbol_attributes_dict, className_wordEmbeddings_dict, className_classSymbol_dict, train_class_symbol, train_image_path_list, test_image_path_list = get_all_data(logger)


2018-09-03 21:50:01,378 config:
{'DEBUG': False,
 'alpha': 0.99,
 'data_loader_num_workers': 8,
 'data_path': '/home/messor/data_center/object_classification/zero_sample_classification',
 'dataset': {'b_g_r_mean': [108.74949161620336,
                            121.93682191782138,
                            129.882933212282],
             'b_g_r_std': [76.27990887272082,
                           72.2227214967396,
                           73.87829834580016],
             'className_classSymbol_list': ['/home/messor/data_center/object_classification/zero_sample_classification/DatasetA_train/label_list.txt'],
             'className_wordEmbeddings_list': ['/home/messor/data_center/object_classification/zero_sample_classification/DatasetA_train/class_wordembeddings.txt',
                                               '/home/messor/data_center/object_classification/zero_sample_classification/DatasetA_test/class_wordembeddings.txt'],
             'classSymbol_attributes_list': ['/home/

In [8]:
train_class_symbol = train_class_symbol[: config.sample_test]
train_image_path_list = train_image_path_list[: config.sample_test]

In [10]:
classSymbol_wordEmbeddings_dict = replace_name_with_symbol(className_wordEmbeddings_dict, className_classSymbol_dict)

In [11]:
train_label = make_label(train_class_symbol, classSymbol_attributes_dict, classSymbol_wordEmbeddings_dict)

In [22]:
def formate_train_class_num_list(train_class_symbol, className_classSymbol_dict):
    train_class_id = []
    classSymbol_id_dict = {}
    for i, className_symbol in enumerate(className_classSymbol_dict):
        classSymbol_id_dict[className_symbol] = i
    return classSymbol_id_dict
classSymbol_id_dict = formate_train_class_num_list(train_class_symbol, className_classSymbol_dict)

In [26]:
train_class_id = []
for class_symbol in train_class_symbol:
    train_class_id.append(classSymbol_id_dict[class_symbol])

In [28]:
def creat_records(dataset_loader, image_path_list, class_symbol_list, output_record_dir):
    writer = tf.python_io.TFRecordWriter(output_record_dir)
    for i in range(len(image_path_list)):
        image = dataset_loader.__getitem__(i)
        image_raw = image.tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'image_raw': _bytes_feature(image_raw),
            'height': _int64_feature(image.shape[0]),
            'width': _int64_feature(image.shape[1]),
            'depth': _int64_feature(image.shape[2]),
            'label':  _int64_feature(class_symbol_list[i])
        }))
        writer.write(example.SerializeToString())
    writer.close


In [29]:
if config.train.split_val is not None:
    temp = list(zip(train_class_id, train_image_path_list, train_label))
    shuffle(temp)
    train_class_id, train_image_path_list, train_label = zip(*temp)
    len_val = int(len(train_label) * config.train.split_val)

    val_image_path_list = train_image_path_list[: len_val]
    val_class_id = train_class_id[:len_val]
    train_label = train_label[len_val:]
    train_image_path_list = train_image_path_list[len_val:]
    val_data_loader = TestDataLoader(config, val_image_path_list, val_class_id)

    if not os.path.exists(config.dataset.trainTFRecord_list[0]):
        os.makedirs(config.dataset.trainTFRecord_list[0])

    val_record_dir = os.path.join(config.dataset.trainTFRecord_list[0], 'val.tfrecords')
    creat_records(val_data_loader, val_image_path_list, val_class_id, val_record_dir)


2018-09-03 22:27:27,127 using 6115 for test


In [30]:
train_data_loader = TrainDataLoader(train_image_path_list, train_label, config)

if not os.path.exists(config.dataset.trainTFRecord_list[0]):
    os.makedirs(config.dataset.trainTFRecord_list[0])
train_record_dir = os.path.join(config.dataset.trainTFRecord_list[0], 'train.tfrecords')
creat_records(train_data_loader, train_image_path_list, train_class_id, train_record_dir)


2018-09-03 22:28:37,724 using 24462 for train
